# 1. async/await

使用async、await完成如下程序： 

主例程等待两个例程结束，其中例程一读取文件 **IBM_2006-01-01_to_2018-01-01.csv** 的单数行，例程二将读取结果写入文件 **new.csv**

In [28]:
%%writefile co-routine.py
import asyncio
import time
import csv
res = []
async def readfile():
    global res
    csv_reader = csv.reader(open("IBM_2006-01-01_to_2018-01-01.csv", "r"), delimiter=',')
    cnt = 0
    for row in csv_reader:
        cnt += 1
        if cnt % 2:
            res.append(row)

async def writefile():
    global res
    writer = csv.writer(open('new.csv', mode='w',newline=""), delimiter=",")
    for i in res:
        writer.writerow(i)
    
async def main():
    print(f"start at {time.strftime('%X')}")
    await readfile()
    await writefile()
    print(f"finish at {time.strftime('%X')}")

asyncio.run(main())

Writing co-routine.py


## 选做

将 **AMZN_2006-01-01_to_2018-01-01.csv** 和 **IBM_2006-01-01_to_2018-01-01.csv** 合并成一个文件 **combine.csv**，推荐通过并发技术提高效率（不限协程）。

In [27]:
%%writefile co-combine.py
import asyncio
import time
import csv

async def readfile(filename):
    res = []
    csv_reader = csv.reader(open(filename, "r"), delimiter=',')
    for row in csv_reader:
        res.append(row)
    return res

async def writefile(lst):
    writer = csv.writer(open('combine.csv', mode='w',newline=""), delimiter=",")
    for i in lst:
        writer.writerow(i)
    
async def main():
    print(f"start at {time.strftime('%X')}")
    task1 = asyncio.create_task(readfile("AMZN_2006-01-01_to_2018-01-01.csv"))
    task2 = asyncio.create_task(readfile("IBM_2006-01-01_to_2018-01-01.csv"))
    lst1 = await task1
    lst2 = await task2
    lst2.pop(0)
    await writefile(lst1+lst2)
    print(f"finish at {time.strftime('%X')}")

asyncio.run(main())

Overwriting co-combine.py


# 2. Yield

使用yield 完成如下程序：
按块读取文件 IBM_2006-01-01_to_2018-01-01.csv，每次返回 BLOKSIZE 行内容

In [42]:
def readfile_block():
    BLOCK_SIZE = 1024
    res = []
    csv_reader = csv.reader(open("IBM_2006-01-01_to_2018-01-01.csv", "r"), delimiter=',')
    cnt = 0
    for row in csv_reader:
        cnt += 1
        res.append(row)
        if cnt % BLOCK_SIZE == 0:
            yield res
            res = []
    yield res
f = readfile_block()

In [43]:
# 文件里共有3021行， 3021/1024上取整为3
cnt = 0
for i in f:
    cnt += 1
print(cnt)

3
